# Serverless

- Name: Aryan Ebrahimpour
- ID: 98726942

In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
#r "nuget: FSharp.Data"
#r "nuget: NodaTime"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

open System
open Plotly.NET
open Plotly.NET.Interactive
open NodaTime
open System.Text.Json
open System.Text.Json.Serialization

Installed Packages FSharp.Data, 4.2.8 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NodaTime, 3.1.0 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
#load "Serverless.fsx"
open Avestura.Serverless
open MathNet.Numerics.Distributions

In [ ]:
let inline aggregateMeassure util cost respT = (util * util) / (cost * respT)

let inline simAggr (sctx: SimulatorContextBatchQoS) =
    let u, _ = sctx.utilization
    let c, _ = sctx.cost
    let r, _ = sctx.responseTime
    aggregateMeassure u c r |> float

In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers

let iterationsCount = 10

let queueD = genQueue config_UFD_AW_N 3u
let simD = runSim queueD AllSchedulers.random240_50MeanWaitScheduler

[simD] |> SimulatorContextBatch.getBatchQoS |> (fun x -> x.responseTime)


Item1,Item2
7.61950159727193,1.9694019777700331


In [ ]:
//simD.containers["container-1"] |> Container.getQosMeassures |> (fun x -> x.Value.responseTime)
simD.containers["container-1"].totalRestorationDuration

Days,NanosecondOfDay,Hours,Minutes,Seconds,Milliseconds,SubsecondTicks,SubsecondNanoseconds,BclCompatibleTicks,TotalDays,TotalHours,TotalMinutes,TotalSeconds,TotalMilliseconds,TotalTicks,TotalNanoseconds
0,306000000000,0,5,6,0,0,0,3060000000,0.0035416666666666665,0.085,5.1,306,306000,3060000000,306000000000


In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers

module Sim1 =
    let iterationsCount = 4

    let queue = genQueue config_UFD_AW_N 3u
    let runForScheduler = runBatch iterationsCount queue
    let noMer, staticSc, randSc, dynNeut, dynCtx = forAllSchedulers runForScheduler
    let q1, q2, q3, q4, q5 = qosOf noMer, qosOf staticSc, qosOf randSc, qosOf dynNeut, qosOf dynCtx

In [ ]:
[Sim1.q1; Sim1.q2; Sim1.q3; Sim1.q4; Sim1.q5]

index utilization responseTime turnaroundTime cost 0 Item1 Item2 61.74528075687604 23.509201144720922 Item1 Item2 22.535521528114955 6.198141867089305 Item1 Item2 86.7078274598348 62.85359970350629 Item1 Item2 10.860155389344301 0.006996154960017213 1 Item1 Item2 51.61189247077901 7.006714493020439 Item1 Item2 9.089379672108434 1.7891826270315823 Item1 Item2 74.51357641133139 12.702438688534446 Item1 Item2 15.734688659508505 0.08364385028642769 2 Item1 Item2 54.031882069299336 8.537568195917391 Item1 Item2 9.715361985687691 2.932896698643093 Item1 Item2 72.59727549670025 14.329372399496311 Item1 Item2 14.911233941274874 0.05104400356260112 3 Item1 Item2 66.51893603127044 21.108164641362286 Item1 Item2 18.375759022108827 7.006367239132188 Item1 Item2 78.39057988157573 48.81201208095265 Item1 Item2 10.651725448119914 0.012857034654843422 4 Item1 Item2 66.78784405972479 21.052594496669027 Item1 Item2 18.284041423921728 7.0022299035637765 Item1 Item2 78.66022570958705 48.27232240312318 Item1 Item2 10.629800503300693 0.010595369974594185

In [ ]:
let range = seq { 0.0..0.1..40. }
[
    Sim1.q1 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim1.q2 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim1.q3 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim1.q4 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim1.q5 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان پاسخ به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..200. }
[
    Sim1.q1 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim1.q2 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim1.q3 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim1.q4 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim1.q5 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان برگشت به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..100. }
[
    Sim1.q1 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim1.q2 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim1.q3 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim1.q4 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim1.q5 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
]
|> Chart.combine
|> Chart.withXAxisStyle "بهره‌وری به درصد"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.001..4. }
let (c1, _), (c2, _), (c3, _), (c4, _), (c5, _) = Sim1.q1.cost, Sim1.q2.cost, Sim1.q3.cost, Sim1.q4.cost, Sim1.q5.cost
let costs = [c1; c2; c3; c4; c5] |> List.map float
let title = ["No Merge"; "Static Wait"; "Random Wait"; "Dynamic Neutral"; "Dynamic Context"]

Chart.Column(costs,title) |> Chart.withTitle "نمودار هزینه"

<!-- Plotly chart will be drawn inside this DIV -->

## for 30 functions

In [ ]:
open SimulationRunnerEngine
open EvaluatorConfigs
open AllSchedulers

module Sim2 =
    let iterationsCount = 4

    let queue = genQueue config_UFD_AW_N 30u
    let runForScheduler = runBatch iterationsCount queue
    let noMer, staticSc, randSc, dynNeut, dynCtx = forAllSchedulers runForScheduler
    let q1, q2, q3, q4, q5 = qosOf noMer, qosOf staticSc, qosOf randSc, qosOf dynNeut, qosOf dynCtx

In [ ]:
[Sim2.q1; Sim2.q2; Sim2.q3; Sim2.q4; Sim2.q5]

index utilization responseTime turnaroundTime cost 0 Item1 Item2 69.32133884101093 22.854355566440553 Item1 Item2 14.80346402089967 6.7494921715680665 Item1 Item2 77.38600537210286 60.756830117816364 Item1 Item2 9.75966504417835 0.009781678642677427 1 Item1 Item2 50.84511416285772 9.904784412255209 Item1 Item2 9.302300815688525 1.49511802516452 Item1 Item2 71.62821677199452 15.983742798681309 Item1 Item2 15.48266868250492 0.06776791231460158 2 Item1 Item2 54.551582108576525 10.417308992939631 Item1 Item2 9.364248566216867 1.8513417714473426 Item1 Item2 71.91086119936737 17.83612116919079 Item1 Item2 14.613335862484409 0.07456274872436473 3 Item1 Item2 72.1986708139608 20.05929092012396 Item1 Item2 12.966827473150293 5.8857131824162945 Item1 Item2 72.67587354437781 47.9727175162563 Item1 Item2 10.13783255047921 0.01309688353516534 4 Item1 Item2 72.20741423474267 19.957260626835478 Item1 Item2 13.038669545778559 5.95186911879948 Item1 Item2 73.13881873654235 48.80446479549815 Item1 Item2 10.132343301740253 0.005533174593162044

In [ ]:
let range = seq { 0.0..0.1..30. }
[
    Sim2.q1 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim2.q2 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim2.q3 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim2.q4 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim2.q5 |> SimulatorContextBatchQoS.responseTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان پاسخ به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..180. }
[
    Sim2.q1 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim2.q2 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim2.q3 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim2.q4 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim2.q5 |> SimulatorContextBatchQoS.turnaroundTimeChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
]
|> Chart.combine
|> Chart.withXAxisStyle "زمان برگشت به ثانیه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.1..100. }
[
    Sim2.q1 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("No Merge")
    Sim2.q2 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Static Wait")
    Sim2.q3 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Random Wait")
    Sim2.q4 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Neutral")
    Sim2.q5 |> SimulatorContextBatchQoS.utilizationChartData range |> Chart.Spline |> Chart.withTraceName ("Dynamic Context")
]
|> Chart.combine
|> Chart.withXAxisStyle "بهره‌وری به درصد"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let range = seq { 0.0..0.001..4. }
let (c1, _), (c2, _), (c3, _), (c4, _), (c5, _) = Sim2.q1.cost, Sim2.q2.cost, Sim2.q3.cost, Sim2.q4.cost, Sim2.q5.cost
let costs = [c1; c2; c3; c4; c5] |> List.map float
let title = ["No Merge"; "Static Wait"; "Random Wait"; "Dynamic Neutral"; "Dynamic Context"]

Chart.Column(costs,title) |> Chart.withTitle "هزینه"

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Visualization of the General.normalize
// function using Weilbul Stretched Function

let x = [ for x in 0. .. 0.1 .. 20 -> (x, General.normalize x) ]

x |> Chart.Line

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Dijestra path finding

let dg = DependencyGraph.init (20u)

(dg.adjMatrix) |> GraphExtensions.printAdjMatrix

let path, distance = dg |> DependencyGraph.dijkestraPath (0u) (5u)
let weights = dg |> DependencyGraph.edgeWeightsFromPath path

path, weights, distance


--------------
|  0 | 909 | 745 | 0 | 637 | 0 | 557 | 0 | 936 | 317 | 209 | 0 | 937 | 0 | 918 | 784 | 0 | 0 | 908 | 453 |
--------------
|  616 | 0 | 787 | 0 | 0 | 781 | 423 | 0 | 0 | 0 | 797 | 0 | 0 | 106 | 738 | 593 | 0 | 49 | 697 | 863 |
--------------
|  344 | 489 | 0 | 198 | 106 | 127 | 0 | 232 | 546 | 0 | 0 | 0 | 960 | 0 | 0 | 220 | 920 | 470 | 0 | 295 |
--------------
|  0 | 0 | 505 | 73 | 0 | 0 | 941 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 722 | 895 | 713 |
--------------
|  426 | 196 | 0 | 763 | 99 | 0 | 0 | 0 | 594 | 601 | 0 | 0 | 0 | 55 | 580 | 0 | 171 | 0 | 0 | 0 |
--------------
|  430 | 0 | 0 | 433 | 994 | 451 | 320 | 767 | 0 | 0 | 0 | 0 | 0 | 596 | 0 | 810 | 325 | 948 | 344 | 819 |
--------------
|  752 | 0 | 0 | 974 | 768 | 809 | 334 | 296 | 0 | 0 | 553 | 955 | 0 | 663 | 237 | 40 | 715 | 54 | 69 | 634 |
--------------
|  492 | 0 | 251 | 0 | 425 | 0 | 0 | 0 | 0 | 0 | 406 | 135 | 681 | 0 | 0 | 60 | 208 | 0 | 825 | 0 |
--------------
|  872 | 831 | 650 | 0 | 985 | 787 | 

In [ ]:
// fsevents
// hiredis
// karma-phantomjs-launcher
// phantomjs-prebuilt
// puppeteer
// serialport
// xml2json
// yeoman-environment
// let dd_impossible = [
//     "fsevents"
//     "hiredis"
//     "karma-phantomjs-launcher"
//     "phantomjs-prebuilt"
//     "puppeteer"
//     "serialport"
//     "xml2json"
//     "yeoman-environment"
// ]
// let dd_1 = PackagesData.dependentUpon.Keys |> List.ofSeq |> List.skipWhile(fun x -> x <> "yeoman-generator")
// PackagesData.downloadAndSaveAllPackageSizes dd_impossible

//dd_1.[1]

In [ ]:
let saturdayData = FrequenciesData.frequencies["saturday"]
Chart.Spline(saturdayData.Keys, saturdayData.Values)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
open QueueDataGenerator
open QueueFunctionGeneration
open Simulator


let config1 : QueueFunctionGeneration.Options = 
    (IgnoreFrequencyData, IgnoreCoopNetwork, defaultUnawareRandomNormalDepsCount)
let queueData = generateFunctionQueueData config1 6
// let context = runSimulatonWithQueueData "saturday" queueData noMergeScheduler

let x = DependencyGraph(queueData.funcs)


Error: input.fsx (13,15)-(13,32) typecheck error This function takes too many arguments, or is used in a context where a function is not expected

In [ ]:
results |> SimulatorContext.getQoSMeassuresList

index utilization responseTime turnaroundTime cost 0 0.04612887905122879 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 522000000000 0 8 42 0 0 0 5220000000 0.0060416666666666665 0.145 8.7 522 522000 5220000000 522000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 528504263147 0 8 48 504 5042631 504263147 5285042631 0.006116947490127315 0.14680673976305555 8.808404385783334 528.504263147 528504.263147 5285042631.47 528504263147 0.0061562505264666665 1 0.06681392075498076 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 288000000000 0 4 48 0 0 0 2880000000 0.0033333333333333335 0.08 4.8 288 288000 2880000000 288000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 290116136176 0 4 50 116 1161361 116136176 2901161361 0.0033578256501851853 0.08058781560444445 4.835268936266667 290.116136176 290116.136176 2901161361.76 290116136176 0.003454308519888889 2 0.0935589204955372 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 343000000000 0 5 43 0 0 0 3430000000 0.003969907407407407 0.09527777777777778 5.716666666666667 343 343000 3430000000 343000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 347575960282 0 5 47 575 5759602 575960282 3475759602 0.004022869910671296 0.09654887785611112 5.792932671366667 347.575960282 347575.960282 3475759602.82 347575960282 0.004260569386766667 3 0.3125057826459409 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 262000000000 0 4 22 0 0 0 2620000000 0.0030324074074074073 0.07277777777777777 4.366666666666666 262 262000 2620000000 262000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 265031932361 0 4 25 31 319323 31932361 2650319323 0.003067499217141204 0.0736199812113889 4.4171988726833336 265.031932361 265031.932361 2650319323.61 265031932361 0.004283380389422223 4 0.27795652702342755 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 249000000000 0 4 9 0 0 0 2490000000 0.0028819444444444444 0.06916666666666667 4.15 249 249000 2490000000 249000000000 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 251073821109 0 4 11 73 738211 73821109 2510738211 0.002905947003576389 0.06974272808583333 4.18456368515 251.073821109 251073.821109 2510738211.09 251073821109 0.0038636304154000003 5 0.03665024188370652 Days NanosecondOfDay Hours Minutes Seconds Milliseconds SubsecondTicks SubsecondNanoseconds BclCompatibleTicks TotalDays TotalHours TotalMinutes TotalSeconds TotalMilliseconds TotalTicks TotalNanoseconds 0 701000000000 0 11 41 0 0 0 70100